---

## Data Analysis

- This file differs from [2_data_analysis_1_base_data.ipynb](2_data_analysis_1_base_data.ipynb) in that it:
    - scales the base cleaned data created in [1_data_cleaning.ipynb](1_data_cleaning.ipynb).

Source dataset: 247076 rows × 37 columns
Processed and analyzed dataset: 247076 rows × 37 columns

---

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib
import config
import time

sys.path.insert(1, config.package_path)
import ml_analysis as mlanlys
import ml_clean_feature as mlclean

In [2]:
start_time = time.time()

---

## 1. Read the cleaned dataset from file

---

In [3]:
# reload any changes to Config Settings
importlib.reload(config)

# BE SURE TO UPDATE THE LABEL FOR THIS ANALYSIS
# #############################
dataset_label = '8 SMOTEENN Dataset'
# #############################

year                        = config.year

clean_file                  = config.clean_file
performance_report          = config.performance_report

report_path                 = config.report_path
file_label                  = dataset_label.lower().replace(' ','_')
detailed_performance_report = report_path + file_label + '_detailed_performance_report.txt'

print(f"Year:                        {year}")
print(f"Clean File:                  {clean_file}")
print(f"Performance Report:          {performance_report}")
print(f"Detailed Performance Report: {detailed_performance_report}")

Year:                        2021
Clean File:                  data/brfss_2021_clean.parquet.gzip
Performance Report:          reports/performance_report.pkl
Detailed Performance Report: reports/8_smoteenn_dataset_detailed_performance_report.txt


In [4]:
# Read final cleaned dataset from parquet file
df = pd.read_parquet(clean_file, engine="fastparquet")

In [5]:
diabetes_labels = df.columns

In [6]:
df.shape

(247076, 37)

---

## 2. Prepare the dataset for analysis

- Split the dataset into features and labels.
- Split the dataset into training and testing sets.
- Scale the dataset

---

In [7]:
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

target = 'diabetes'
# Dictionary defining modification to be made to the base dataset
operation_dict = {  'target_column'     :  target,
                    'convert_to_binary' :  True,
                    'scaler'            : 'standard', # options: none, standard, minmax
                    'random_sample'     : 'smoteen'   # options: none, undersample, oversample
                    }

# This insures that df if not modified during the call to modify_base_dataset()
df_modified = df.copy()

# Modify the base dataset
# data is returned where: X_train, X_test, y_train, y_test = data
data = mlanlys.modify_base_dataset(df_modified, operation_dict)

Base Dataset Modifications in Process
-------------------------------------
**Operation:target_column  diabetes
**Operation:convert_to_binary  True
  -- Converting dataset to binary (0,1) from (0,1,2)


****Cleaning Feature: diabetes
  Initial Unique features in [diabetes]:  [0. 1. 2.]
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: ********* NO Parameters were specified *********
  FINAL Unique features in [diabetes]:  [0. 1.]
**Operation:scaler  standard
  -- Performing train_test_split on dataframe with target:'diabetes'
     -- Run automatically before scalar or random_sample operations
  -- Performing StandardScaler on X_train: Updates X_train, y_test
**Operation:random_sample  smoteen

Dataframe, Train Test Summary
-----------------------------
Dataframe: (247076, 37)  Data:4, X_train:185307, y_train:185307, X_test:61769, y_test:61769
ValueCounts:   y_train: len:2   0: 160547   1: 24760
ValueCounts:   y_test : len:2   0:  5349

In [9]:
# Print some statistics about the original df and the modified dataframe
print(f"Original Dataframe")
print(f"------------------")
print(f"df.shape: {df.shape}")
print(f"df[{target}].value_counts:  {df[target].value_counts()}")

print(f"\nModified Dataframe")
print(f"------------------")
print(f"df_modified.shape: {df_modified.shape}")
print(f"df_modified[{target}].value_counts:  {df_modified[target].value_counts()}")

Original Dataframe
------------------
df.shape: (247076, 37)
df[diabetes].value_counts:  diabetes
0.0    208389
2.0     33033
1.0      5654
Name: count, dtype: int64

Modified Dataframe
------------------
df_modified.shape: (247076, 37)
df_modified[diabetes].value_counts:  diabetes
0.0    214043
1.0     33033
Name: count, dtype: int64


In [10]:
X_train, X_test, y_train, y_test = data
print(f"Dataframe: {df_modified.shape}  Data:{len(data)}, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")
y_train.value_counts()

Dataframe: (247076, 37)  Data:4, X_train:185307, y_train:185307, X_test:61769, y_test:61769


diabetes
0.0    160547
1.0     24760
Name: count, dtype: int64

---

## 3. Run initial Tests and get k_value

**From step 2:**  Data = [X_train_modified, X_test_modified, y_train_modified, y_test]

---

In [11]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

# Determine the k_value
# mlanlys.knn_plot(data)

<module 'ml_analysis' from '/mnt/c/ML/DU/repos/projects/project-2/DU-project-2-2015/brfss_2021/../pkgs/ml_analysis.py'>

**Note:** From the knn plot above, pick a k-value of 3.

---

## 4. Run the Analysis

---

#### Model Run Times

-  Base dataset (247076 rows × 37 columns):

| Model | Run Time |
| ----- | -------- |
| test_model(SVC(kernel='linear'), data)                          | Aborted >35min (Data too large, consider for RandomUndersampling dataset) |
| test_model(KNeighborsClassifier(n_neighbors=k_value), data)     | 247.13 seconds |
| test_model(tree.DecisionTreeClassifier(), data)                 |   3.89 seconds |
| test_model(RandomForestClassifier(), data)                      |  60.94 seconds |
| test_model(ExtraTreesClassifier(random_state=1), data)          |  58.54 seconds |
| test_model(GradientBoostingClassifier(random_state=1), data)    | 115.21 seconds |
| test_model(AdaBoostClassifier(random_state=1), data)            |  11.91 seconds |
| test_model(LogisticRegression(), data)                          |   4.90 seconds |
| **Total** w/o SVC| 502.52 seconds / **8:23 minutes** |

In [12]:
# reload any changes to nlanlys
importlib.reload(mlanlys)

k_value = 3

#### COMMENT OUT ONE OF THE FOLLOWING SECTIONS

## SECTION 1
# Capture stdout & stderr into two strings: osc.stdout and osc.stderr that contain the output from the function
# -- This allows the output to be printed here or to a file or both.

with mlanlys.OutStreamCapture() as osc:
    performance_summary = mlanlys.run_classification_models(data, k_value)
#    performance_summary = mlanlys.run_classification_models_test(data, k_value)

## <OR>
## SECTION 2

# performance_summary = mlanlys.run_classification_models(data, k_value)


In [13]:
# UNCOMMENT if using SECTION 1 in the previous step
# print(osc.stdout)

# Add code to print osc.stdout to a file if desired.

---

### 4.1 Archive Performance Summary

- For use in Project-2 Performance Summary Report
---

In [14]:
# performance_summary is a dataframe of performance statistics

# Add the dataset label as the first column in performance_summary
dataset_column = pd.Series([dataset_label] * len(performance_summary), name=dataset_label)
performance_summary.insert(0, 'new_column', dataset_column)

analysis_perf_summary = { 'dataset_size': list(df.shape), 'report': performance_summary}

# Performance_report is a file containing all the performance summary statistics
if os.path.exists(performance_report):
    print(f"The file {performance_report} exists.")
    # Load Performance Report
    with open(performance_report, 'rb') as file: perf_report = pickle.load(file)
else:
    print(f"The file {performance_report} does not exist.")
    perf_report = {}
    
perf_report[dataset_label] = analysis_perf_summary

# Save Performance Report
with open(performance_report, 'wb') as file: pickle.dump(perf_report, file)

The file reports/performance_report.pkl exists.


### 4.2 Archive the Performance Detailed Statistics Report
---

In [15]:
# osc.stdout contains the details of the performance statistics

with open(detailed_performance_report, "w") as file:
    file.write(osc.stdout)

---

## 5. Performance Summary

---

In [16]:
# print the performance summary
print(f"******************************************")
print(f"Performance Summary for: {dataset_label}")
print(f"******************************************")

performance_summary

******************************************
Performance Summary for: 8 SMOTEENN Dataset
******************************************


,new_column,model,slice,score,balanced_accuracy,roc_auc_score,Mean Squared Error,Accuracy,Precision,Recall,F1-score,Specificity,False Positive Rate,Matthews Correlation Coefficient
0,8 SMOTEENN Dataset,KNeighborsClassifier,Train,0.9053,0.7236,0.9352,0.0947,0.9053,0.7209,0.4755,0.5730,0.9716,0.0284,NaN
1,8 SMOTEENN Dataset,KNeighborsClassifier,Test,0.8426,0.5849,0.6819,0.1574,0.8426,0.3634,0.2328,0.2838,0.9369,0.0631,0.2064
2,8 SMOTEENN Dataset,DecisionTreeClassifier,Train,1.0000,1.0000,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000,NaN
3,8 SMOTEENN Dataset,DecisionTreeClassifier,Test,0.8037,0.6011,0.6011,0.1963,0.8037,0.2910,0.3243,0.3067,0.8778,0.1222,0.1932
4,8 SMOTEENN Dataset,RandomForestClassifier,Train,1.0000,0.9998,1.0000,0.0000,1.0000,1.0000,0.9997,0.9998,1.0000,0.0000,NaN
5,8 SMOTEENN Dataset,RandomForestClassifier,Test,0.8711,0.5570,0.8249,0.1289,0.8711,0.5870,0.1280,0.2102,0.9861,0.0139,0.2307
6,8 SMOTEENN Dataset,ExtraTreesClassifier,Train,1.0000,1.0000,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000,NaN
7,8 SMOTEENN Dataset,ExtraTreesClassifier,Test,0.8697,0.5611,0.8189,0.1303,0.8697,0.5543,0.1395,0.2229,0.9827,0.0173,0.2305
8,8 SMOTEENN Dataset,GradientBoostingClassifier,Train,0.8731,0.5739,0.8402,0.1269,0.8731,0.5891,0.1656,0.2585,0.9822,0.0178,0.2645
9,8 SMOTEENN Dataset,GradientBoostingClassifier,Test,0.8723,0.5744,0.8356,0.1277,0.8723,0.5810,0.1674,0.2599,0.9813,0.0187,0.2630


In [17]:
print(f"Completed: Execution Time %s seconds:" % round((time.time() - start_time),2) )

Completed: Execution Time 572.08 seconds:


---

## 6. Conclusions

- A first glance at the summary, it appears that the Boosting models may have performed well with test/train scores were >.8 and similar in scale (<.02 delta).  However, the poor test confusion matrix and balanced accuracy highlight the overfitting.

- The Base Cleaned data is overfit as indicated by:
    - Poor confusion matrix on the detailed report for test sets on all models
    - Low balanced accuracy as compared to the model score (less than 50%)


---

---